In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# googleplaystore.csv: https://drive.google.com/file/d/1o1jWeOlOz9r2Q6JqBhckUKbZRuBrhqKJ/view?usp=drive_link
# googleplaystore_user_reviews.csv: https://drive.google.com/file/d/1XX4S3LUe_bVB94g5H2tv5rhEANeSX_sO/view?usp=drive_link
downloaded = drive.CreateFile({'id':"1o1jWeOlOz9r2Q6JqBhckUKbZRuBrhqKJ"})
downloaded.GetContentFile('googleplaystore.csv')

In [ ]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context -> variable que vamos a utilizar
sc = spark.sparkContext

Obtenga la matriz de distancias en días entre fechas de actualización de las aplicaciones pagas. ¿Cuáles son las dos aplicaciones con mayor distancia? (⭐⭐)

In [ ]:
sqlContext = SQLContext(sc)
df_googleplaystore = sqlContext.read.option("delimiter", ",").option("escape", '"').csv('googleplaystore.csv', header=True, inferSchema=True)
rdd = df_googleplaystore.rdd
rdd = rdd.filter(lambda x: x['Type'] == 'Paid')
rdd = rdd.map(lambda x: (x['App'], pd.to_datetime(x['Last Updated'])))
matriz = rdd.cartesian(rdd)
distancias = matriz.map(lambda x: (x[0][0], x[1][0], (x[0][1] - x[1][1]).days))
mayor_dist = distancias.reduce(lambda x, y: x if x[2] > y[2] else y)
print(f"Las dos aplicaciones con mayor distancia son {mayor_dist[0]} y {mayor_dist[1]}")

Las dos aplicaciones con mayor distancia son Fast Tract Diet y AH Alarm Panel
